<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JSON" data-toc-modified-id="JSON-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JSON</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li></ul></div>

# JSON

JSON(Javascript Object Notation) : Javascript에서 객체를 표현하는 문법을 이용해 데이터를 표현하는 문서의 양식

- 정수 : 100
- 실수 : 11.11
- Boolean : true,false
- 문자열 : '문자열'
- 배열 : [] => python list
- 객체 : {} => python dictionary

In [23]:
import requests
import json
import re
import time
import pandas as pd

In [24]:
chk = False

In [25]:
start = 1
display = 100

# 검색어를 가져온다.
keyword = getKeyword()

while True :
    
    jason_obj = connectionSite(keyword,start,display)    
    total = getData(jason_obj)
    
    print(f'{start}~{start+100}/{total}')

    # 검색결과가 더 있는지 확인
    start = start+100
    if start >= total:
        break
    elif start == 1001:
        start = 1000
    elif start == 1100:
        break

검색어를 입력해주세요 : 캡틴마블
1~101/30808
101~201/30808
201~301/30808
301~401/30808
401~501/30808
501~601/30808
601~701/30808
701~801/30808
801~901/30808
901~1001/30808
1000~1100/30808


# Functions

In [11]:
# 전달된 모든 문자열에서 html 태그를 제거한다.
def getText(data_str):
    data_str = re.sub('<.+?>','',data_str).strip() 
    return data_str

In [12]:
def getKeyword():
    keyword = input('검색어를 입력해주세요 : ')
    return keyword

In [13]:
def connectionSite(Keyword,start,display):
    #요청할 URL
    url =  'https://openapi.naver.com/v1/search/blog.json'
    
    #parameters
    params = {
        'query': Keyword,
        'start' : start,
        'display' : display
    }

    #헤더정보
    headers = {
        'X-Naver-Client-Id':'IS3OiCTNiOI4ohZPsBRx',
        'X-Naver-Client-Secret':'Nxmj3sNiJZ'
    }

    #요청 json
    res = requests.get(url,params = params, headers = headers)
    
    # json 객체를 dictionary 나 list 로 바꾼다.
    # requests.text가 {} 로 묶여있으면 dictionary, []로 묶여있으면 list 다.
    json_dict = json.loads(res.text)
    
    return json_dict

In [22]:
def getData(json_obj):
    
    # data frame을 저장했는지 여부를 확인
    global chk

    #item 태그를 가져온다.
    
    for item in jason_obj['items']:
        title_str = getText(item['title'])
        desc_str = getText(item['description'])
        bloggername_str = getText(item['bloggername'])
        bloggerlink_str = getText(item['bloggerlink'])
        date_str = getText(item['postdate'])
        link_str = getText(item['link'])

        data_dic = {
            'title':[title_str],
            'decription':[desc_str],
            'link':[link_str],
            'bloggername':[bloggername_str],
            'bloggerlink':[bloggerlink_str],
            'postdate':[date_str]
        }
        
        df = pd.DataFrame(data_dic)

        if chk == False:
            chk = True
            df.to_csv(f'{keyword}.csv',index = False, encoding = 'utf-8-sig')
        else :
            df.to_csv(f'{keyword}.csv',index = False, encoding = 'utf-8-sig', mode = 'a',header = False)

        #total 
        total = jason_obj['total']
        
    return int(total)